In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch

In [2]:
import re #thư viện dùng xử lý chuỗi
import unicodedata #thư viện dùng decode câu thành unicode8
import spacy #thư viện giúp xử lý, trích xuất thông tin từ văn bản 
from tqdm import tqdm #thư viện dùng để tạo thanh triến trình (cho train model)
import pandas as pd #thư viện dùng để tạo ma trận dữ liệu
import numpy as np #thư viện dùng để xử lý mảng nhiều chiều
import random #thư viện tạo số ngẫu nhiên
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator #vẽ biểu đồ wordcloud
import matplotlib.pyplot as plt #vẽ biểu đồ phân tích data
import seaborn as sns #vẽ biểu đồ phân tích data

# GEC_Baseline_Attention

In [3]:
import re
import datetime
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import keras
import tensorflow as tf
from sklearn.metrics import fbeta_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import sentence_bleu
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv("cleaned_data.csv") #đọc lại file csv đã lưu

In [5]:
data.shape

(1398232, 3)

In [6]:
data = data[:500000]

In [7]:
def preprocess(t, add_start_token, add_end_token):
  t = str(t)
  if add_start_token == True and add_end_token == False: #nếu thêm start token và không thêm end token
    t = '<start>'+' '+t
  if add_start_token == False and add_end_token == True: #nếu không thêm start token và thêm end token
    t = t+' '+'<end>'
  if add_start_token == True and add_end_token == True: #nếu thêm cả start token và end token
    t = '<start>'+' '+t+' '+'<end>'

  t = re.sub(' +', ' ', t) #loại bỏ khoảng trắng thừa
  return t

In [8]:
encoder_input = [preprocess(line, add_start_token= True, add_end_token=True) for line in data['error']]
decoder_input = [preprocess(line, add_start_token= True, add_end_token=False) for line in data['correct']]
decoder_output = [preprocess(line, add_start_token= False, add_end_token=True) for line in data['correct']]

In [9]:
print(encoder_input[0]) # in ra dòng đầu tiên của encoder_input
print(decoder_input[0]) 
print(decoder_output[0]) #output của ecoder output
print(len(encoder_input), len(decoder_input), len(decoder_output)) #số lượng dòng trong 3 cột

<start> bitcoin is for , this morning , which coindesk says . <end>
<start> bitcoin goes for , this morning , according to coindesk .
bitcoin goes for , this morning , according to coindesk . <end>
500000 500000 500000


In [10]:
print(data.shape)

(500000, 3)


In [13]:
tokenizer = Tokenizer(filters='', split=" ")
tokenizer.fit_on_texts(encoder_input)
word_index = tokenizer.word_index #tạo index cho token

max_length = max([ len(row.split(" ")) for row in encoder_input ]) #tìm độ dài lớn nhất của câu trong encoder_input
INPUT_ENCODER_LENGTH = max_length 

enc_input_encoded = tokenizer.texts_to_sequences(encoder_input) #tokenize các từ thành dạng index
enc_input_padded= pad_sequences(enc_input_encoded, maxlen=INPUT_ENCODER_LENGTH, padding="post") #đệm các câu để có cùng độ dài thành câu lớn nhất

print(enc_input_padded.shape)
#lưu tokenizer
import pickle
with open('in_tokenizer_fastext.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

(500000, 32)


In [15]:
#kiểm tra id token start và end
print("ID của token <start>:", tokenizer.word_index['<start>'])
print("ID của token <end>:", tokenizer.word_index['<end>'])

ID của token <start>: 1
ID của token <end>: 2


In [23]:
print(INPUT_ENCODER_LENGTH)

32


In [24]:
print(encoder_input[0]) #in ra câu đầu tiên của encoder_input
print(enc_input_padded[0]) #sau khi được chuyển sang index và padding

<start> bitcoin is for , this morning , which coindesk says . <end>
[    1  3466    11    13     5    22   793     5    54 64899   493     3
     2     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [14]:
decoder_data = decoder_input.copy() #copy decoder_input sang decoder_data
decoder_data.extend(decoder_output) #nối decoder_output vào cuối decoder_data

out_tokenizer = Tokenizer(filters='', split=" ") #tạo tokenizer cho decoder_data
out_tokenizer.fit_on_texts(decoder_data) #fit dữ liệu vào tokenizer
word_index = out_tokenizer.word_index #tạo index cho từ

max_length = max([ len(row.split(" ")) for row in decoder_input ]) #tìm độ dài lớn nhất của câu trong decoder_input
INPUT_DECODER_LENGTH = max_length
with open('out_tokenizer_fastext.pkl', 'wb') as f:
    pickle.dump(out_tokenizer, f) #lưu tokenizer

In [17]:
print("ID của token <start>:", out_tokenizer.word_index['<start>'])
print("ID của token <end>:", out_tokenizer.word_index['<end>'])

ID của token <start>: 4
ID của token <end>: 5


In [26]:
dec_input_encoded = out_tokenizer.texts_to_sequences(decoder_input) #chuyển các từ thành index
dec_input_padded= pad_sequences(dec_input_encoded, maxlen=INPUT_DECODER_LENGTH, padding="post", truncating = "post") #đệm các câu để có cùng độ dài thành câu lớn nhất

print(dec_input_padded.shape)

(500000, 32)


In [27]:
print(decoder_input[0]) #in ra câu đầu tiên của decoder_input
print(dec_input_padded[0]) #sau khi được chuyển sang index và padding

<start> bitcoin goes for , this morning , according to coindesk .
[    4  3332  1168    12     3    23   802     3   434     7 57414     1
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [21]:
dec_output_encoded = out_tokenizer.texts_to_sequences(decoder_output) #chuyển các từ thành index
dec_output_padded= pad_sequences(dec_output_encoded, maxlen=INPUT_DECODER_LENGTH, padding="post", truncating = "post") #đệm các câu để có cùng độ dài thành câu lớn nhất

print(dec_output_padded.shape)

(500000, 32)


In [29]:
print(decoder_output[1])
print(dec_output_padded[1])

 . the effect of widespread dud targets two face up attack position monsters on the field . <end>
[    1     2  1147     8  7715 57415  3795   106   708    51  1874   894
  8241    15     2   513     1     5     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [30]:
import io #thư viện dùng để xử lý file

def load_vectors(fname): #hàm load vector từ file
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore') #mở file
    n, d = map(int, fin.readline().split()) #đọc số lượng từ và số chiều của từ
    data = {} #tạo dictionary lưu từ và vector
    for line in fin: #đọc từng dòng trong file
        tokens = line.rstrip().split(' ') #tách từng từ và vector
        data[tokens[0]] = np.asarray(tokens[1:]) #lưu từ và vector vào dictionary
    return data

In [ ]:
in_embedding_matrix = np.load(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\in_embedding.npy")
out_embedding_matrix = np.load(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\out_embedding.npy")
print(in_embedding_matrix.shape, out_embedding_matrix.shape)

(310385, 300) (209334, 300)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Encoder(nn.Module):
    def __init__(self, inp_vocab_size, embedding_size, lstm_size, input_length):
        super(Encoder, self).__init__()
        self.vocab_size = inp_vocab_size #số lượng từ vựng
        self.embedding_size = embedding_size
        self.lstm_units = lstm_size
        self.input_length = input_length

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(in_embedding_matrix), freeze=True,padding_idx=0)
        self.lstm = nn.LSTM(self.embedding_size, self.lstm_units,batch_first=True)
    def forward(self,input_sequence,states):
        input_embedding = self.embedding(input_sequence)
        self.lstm_output, (self.state_h, self.state_c) = self.lstm(input_embedding, states)
        return self.lstm_output, self.state_h, self.state_c
    def initialize_states(self,batch_size):
        lstm_state_h = torch.zeros((1,batch_size,self.lstm_units)).to(device)
        lstm_state_c = torch.zeros((1,batch_size,self.lstm_units)).to(device)
        return lstm_state_h, lstm_state_c

In [6]:
def dot_func(encoder_output,decoder_hidden_state):
    decoder_hidden_state = torch.reshape(decoder_hidden_state,[decoder_hidden_state.shape[1],1,decoder_hidden_state.shape[2]])
    dot_product = torch.matmul(encoder_output,decoder_hidden_state.transpose(1,2))
    return dot_product

In [7]:
class Attention(nn.Module):
    def __init__(self,scoring_function, att_units):
        super(Attention,self).__init__()
        self.scoring_function = scoring_function
        self.att_units = att_units
        self.timesteps = 0
    def forward(self,decoder_hidden_state,encoder_output):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        encoder_output = encoder_output.to(device)
        decoder_hidden_state = decoder_hidden_state.to(device)
        if self.scoring_function == 'dot':
            alpha = F.softmax(dot_func(encoder_output,decoder_hidden_state),dim = 1)
            c_t = torch.sum(alpha *encoder_output,dim = 1)
            return c_t,alpha


In [8]:
class One_Step_Decoder(nn.Module):
    def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super(One_Step_Decoder,self).__init__()

        self.tar_vocab_size = tar_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(out_embedding_matrix), freeze=True,padding_idx=0)
        self.lstm = nn.LSTM(self.dec_units + self.embedding_dim,self.dec_units,batch_first=True)
        self.dense = nn.Linear(self.dec_units,self.tar_vocab_size)
    def forward(self,input_to_decoder,encoder_output,state_h,state_c):
        input_embedding = self.embedding(input_to_decoder)
        if self.score_fun == "dot":
            attention = Attention("dot",self.att_units)
            context_vector,attention_weights = attention(state_h,encoder_output)
        out = torch.cat([input_embedding,context_vector.unsqueeze(1)],dim = 2)
        self.lstm_output, (self.state_h, self.state_c_) = self.lstm(out, (state_h,state_c))
        result_out = self.dense(self.lstm_output)
        return result_out.squeeze(1), self.state_h, self.state_c_, attention_weights, context_vector

In [9]:
class Decoder(nn.Module):
    def __init__(self,out_vocab_size,embedding_dim,input_length,dec_units, score_fun,att_units):
        super(Decoder,self).__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units
        self.one_step_decoder = One_Step_Decoder(self.out_vocab_size, self.embedding_dim, self.input_length, self.dec_units ,self.score_fun ,self.att_units)
    def forward(self,input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_to_decoder = input_to_decoder.to(device)
        encoder_output = encoder_output.to(device)
        decoder_hidden_state = decoder_hidden_state.to(device)
        decoder_cell_state = decoder_cell_state.to(device)

        batch_size, timesteps = input_to_decoder.shape
        out_array = torch.zeros((batch_size, timesteps, self.one_step_decoder.tar_vocab_size)).to(device)

        for timestep in range(timesteps):
            output,decoder_hidden_state,decoder_cell_state,_,_ = self.one_step_decoder(input_to_decoder[:,timestep:timestep+1],encoder_output,decoder_hidden_state,decoder_cell_state)
            out_array[:,timestep,:] = output
        return out_array

In [10]:
class encoder_decoder(nn.Module):
    def __init__(self,enc_units,dec_units,scoring_func,att_units):
        super(encoder_decoder,self).__init__()
        self.scoring_func = scoring_func
        self.att_units = att_units
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.encoder = Encoder(INPUT_VOCAB_SIZE, embedding_size = 300, lstm_size= self.enc_units , input_length= INPUT_ENCODER_LENGTH)
        self.decoder = Decoder(OUTPUT_VOCAB_SIZE, embedding_dim=300, input_length = None, dec_units= self.dec_units ,score_fun =self.scoring_func,att_units = self.att_units)
    
    def forward(self,data):
        input,output = data[0],data[1]
        states = self.encoder.initialize_states(input.shape[0])
        encoder_output,encoder_final_state_h,encoder_final_state_c = self.encoder(input, states)
        decoder_output = self.decoder(output,encoder_output, encoder_final_state_h, encoder_final_state_c)
        return decoder_output

In [22]:
INPUT_VOCAB_SIZE = len(list(tokenizer.word_index)) +2 #kích thước từ vựng đầu vào +2 là do thêm start token và end token
OUTPUT_VOCAB_SIZE = len(list(out_tokenizer.word_index)) +2 #tương tự như trên
print(INPUT_VOCAB_SIZE, INPUT_ENCODER_LENGTH, OUTPUT_VOCAB_SIZE, INPUT_DECODER_LENGTH) #in ra kích thước tập từ vựng, đọ dài câu, kích thước tập từ vựng đầu ra, độ dài câu đầu ra, kích thước batch

310385 32 209334 32


In [23]:
import torch
import torch.nn as nn
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self,enc_input_padded,dec_input_padded,dec_output_padded):
        self.enc_input_padded = torch.tensor(enc_input_padded)
        self.dec_input_padded = torch.tensor(dec_input_padded)
        self.dec_output_padded = torch.tensor(dec_output_padded)
    def __len__(self):
        return len(self.enc_input_padded)
    def __getitem__(self,idx):
        return self.enc_input_padded[idx],self.dec_input_padded[idx],self.dec_output_padded[idx]


dataset = Seq2SeqDataset(enc_input_padded,dec_input_padded,dec_output_padded)
test_size = int(len(dataset)*0.3)
train_size = len(dataset) - test_size
train_dataset,test_dataset = torch.utils.data.random_split(dataset,[train_size,test_size])

#lưu training và test dataset


NameError: name 'dec_input_padded' is not defined

In [40]:
torch.save(train_dataset, 'train_dataset.pt')
torch.save(test_dataset, 'test_dataset.pt')

In [24]:
#load training và test dataset
train_dataset = torch.load('train_dataset.pt')
test_dataset = torch.load('test_dataset.pt')

C:\Users\Admin\AppData\Local\Temp\ipykernel_18572\4009569104.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load('train_dataset.pt')
C:\Users\Admi

In [25]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 32,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = 32,shuffle = False)

In [26]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [27]:
for a,b,c in train_loader:
    print(a.shape)
    print(b.shape)
    print(c.shape)
    break
#thử cho encoder_input_padded đi qua lớp embedding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a,b,c = next(iter(train_loader))
a = torch.randn(32, 32, 256, device=device)  # PHẢI có dòng này
b = torch.randint(0, 1000, (32,1 ,256)).to(device).float()
c= torch.matmul(a,b.transpose(1,2))
print("Alo", c.shape)
alpha = F.softmax(c, dim=1)
print(torch.sum(alpha * a, dim=1).shape)
#thử lớp LSTM
embedding = nn.Embedding.from_pretrained(torch.FloatTensor(in_embedding_matrix), freeze=True,padding_idx=0).to(device)
a = torch.randint(0, 1000, (32,32)).to(device).long()
print(embedding(a).shape)
lstm = nn.LSTM(300,256,batch_first=True).to(device)
lstm_output, (state_h, state_c) = lstm(embedding(a))
print(lstm_output.shape, state_h.shape, state_c.shape)

context_vecto = torch.randn(size = (32, 256)).to(device)
context_vecto = context_vecto.unsqueeze(1)
print(context_vecto.shape)
input_embedding = torch.randn(size = (32, 1, 300)).to(device)
print(torch.cat([input_embedding,context_vecto],dim =2).shape)

torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
Alo torch.Size([32, 32, 1])
torch.Size([32, 256])
torch.Size([32, 32, 300])
torch.Size([32, 32, 256]) torch.Size([1, 32, 256]) torch.Size([1, 32, 256])
torch.Size([32, 1, 256])
torch.Size([32, 1, 556])


In [ ]:
model = encoder_decoder(enc_units=256,dec_units=256,scoring_func="dot",att_units=256)
model.load_state_dict(torch.load(r"C:\Users\Admin\Downloads\lang8model\Model_NLP\NLP\len30_checkpoint.pt")) #đã train được 5 + 1 epoch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()

C:\Users\Admin\AppData\Local\Temp\ipykernel_18572\2617672919.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"C:\Users\Admin\Downloads\

encoder_decoder(
  (encoder): Encoder(
    (embedding): Embedding(310385, 300, padding_idx=0)
    (lstm): LSTM(300, 256, batch_first=True)
  )
  (decoder): Decoder(
    (one_step_decoder): One_Step_Decoder(
      (embedding): Embedding(209334, 300, padding_idx=0)
      (lstm): LSTM(556, 256, batch_first=True)
      (dense): Linear(in_features=256, out_features=209334, bias=True)
    )
  )
)

In [29]:
#tổng tham số của mô hình
total_params = sum(p.numel() for p in model.parameters())

#tổng tham số huấn luyện
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Tham số của mô hình: {total_params}")
print(f"Tham số huấn luyện: {trainable_params}")

Tham số của mô hình: 211119466
Tham số huấn luyện: 55203766


In [32]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, path='len30_checkpoint.pt'):
        self.patience = patience
        self.delta = delta
        self.path = path
        self.best_loss = float('inf')
        self.counter = 0

    def __call__(self, val_loss, model,epoch):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), self.path)  # Lưu model tốt nhất
            print(f"✅ Model từ epoch {epoch+1} đã được lưu tại {self.path}.")
        else:
            self.counter += 1
            print(f"❌ Model từ epoch {epoch+1} không được lưu.")
            if self.counter >= self.patience:
                print("Early stopping triggered!")
                return True
        return False

In [36]:
#đăng nhập vào wandb
import wandb
wandb.login(key = "1a1e9d904fb11812f635b8c3f9a93ae09da4cd04")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc
wandb: Currently logged in as: quoc-trieu-geckokidz (quoc-trieu-geckokidz-hust) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init(project="project_nlp_c4_fromepoch0", name="first_run", config={"epochs": 10, "lr": 1e-3,"time":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")})

In [38]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm  # Import tqdm để hiển thị tiến trình

def train_model(model, train_loader, val_loader, num_epochs=20, lr=1e-3, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)
    early_stopping = EarlyStopping(patience=patience)
    loss_function = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0

        # Thanh tiến trình cho Training
        train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", leave=False)

        for enc_input_padded, dec_input_padded, dec_output_padded in train_progress:
            optimizer.zero_grad()

            enc_input_padded = enc_input_padded.to(device)
            dec_input_padded = dec_input_padded.to(device)
            dec_output_padded = dec_output_padded.to(device)

            outputs = model([enc_input_padded, dec_input_padded])
            outputs = outputs.view(-1, outputs.size(-1))  # (batch_size * seq_len, vocab_size)
            targets = dec_output_padded.view(-1)  # (batch_size * seq_len)

            loss = loss_function(outputs.float(), targets.long())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_progress.set_postfix(loss=loss.item())
        train_loss /= len(train_loader)

        # ----- Validation -----
        model.eval()
        val_loss = 0

        val_progress = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", leave=False)

        with torch.no_grad():
            for enc_input_padded, dec_input_padded, dec_output_padded in val_progress:
                enc_input_padded = enc_input_padded.to(device)
                dec_input_padded = dec_input_padded.to(device)
                dec_output_padded = dec_output_padded.to(device)

                outputs = model([enc_input_padded, dec_input_padded])
                outputs = outputs.view(-1, outputs.size(-1))
                targets = dec_output_padded.view(-1)

                loss = loss_function(outputs.float(), targets.long())
                val_loss += loss.item()
                val_progress.set_postfix(val_loss=loss.item())

        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

        # Log thông số lên WandB
        wandb.log({"epoch": epoch+1, "train_loss": train_loss, "val_loss": val_loss, "learning_rate": optimizer.param_groups[0]["lr"]})

        scheduler.step(val_loss)

        # Lưu mô hình nếu có cải thiện
        if early_stopping(val_loss, model, epoch):
            wandb.run.summary["best_val_loss"] = val_loss
            torch.save(model.state_dict(), "len30_best_model.pth")
            wandb.save("best_model.pth")
            break

    # Kết thúc phiên làm việc với WandB
    wandb.finish()

In [ ]:
#train model
train_model(model, train_loader, test_loader, num_epochs=10, lr=1e-3, patience=5)

Epoch 1/20 - Train Loss: 1.7783 - Val Loss: 2.0109
✅ Model từ epoch 1 đã được lưu tại len30_checkpoint.pt.


Epoch 2/20 - Train Loss: 1.5960 - Val Loss: 1.9909
✅ Model từ epoch 2 đã được lưu tại len30_checkpoint.pt.


Epoch 3/20 - Train Loss: 1.4974 - Val Loss: 1.9895
✅ Model từ epoch 3 đã được lưu tại len30_checkpoint.pt.


Epoch 4/20 - Train Loss: 1.4426 - Val Loss: 1.9812
✅ Model từ epoch 4 đã được lưu tại len30_checkpoint.pt.


Epoch 5/20 - Train Loss: 1.4138 - Val Loss: 1.9888
❌ Model từ epoch 5 không được lưu.


KeyboardInterrupt: 

In [40]:
torch.save(model.state_dict(),"last_model_28_05.pth") #lưu model cuối cùng

In [ ]:
model.load_state_dict(torch.load(r"C:\Users\Admin\Downloads\NLP\best_model.pth"))
model.to(device)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22052\2455360464.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"C:\Users\Admin\Downloads\

encoder_decoder(
  (encoder): Encoder(
    (embedding): Embedding(52381, 300, padding_idx=0)
    (lstm): LSTM(300, 512, batch_first=True)
  )
  (decoder): Decoder(
    (one_step_decoder): One_Step_Decoder(
      (embedding): Embedding(41061, 300, padding_idx=0)
      (lstm): LSTM(812, 512, batch_first=True)
      (dense): Linear(in_features=512, out_features=41061, bias=True)
    )
  )
)

In [ ]:
model.eval()

encoder_decoder(
  (encoder): Encoder(
    (embedding): Embedding(52381, 300, padding_idx=0)
    (lstm): LSTM(300, 512, batch_first=True)
  )
  (decoder): Decoder(
    (one_step_decoder): One_Step_Decoder(
      (embedding): Embedding(41061, 300, padding_idx=0)
      (lstm): LSTM(812, 512, batch_first=True)
      (dense): Linear(in_features=512, out_features=41061, bias=True)
    )
  )
)